In [1]:
from mqt.bench import CompilerSettings, QiskitSettings, get_benchmark
from qiskit import *
import numpy as np

min_qubit = 3 # maximum number of qubit
max_qubit = 5 # minimum number of qubit
gaps = 1
min_qubit = min(max(2, min_qubit), max_qubit) # maximum number of qubit
max_qubit = max(2, max_qubit) # minimum number of qubit
#skip_qubis = max(1, skip_qubits)
gaps = 1
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
for i in range(min_qubit, max_qubit, gaps):
    qc = get_benchmark(benchmark_name="vqe",
                       level="nativegates",
                       circuit_size=i,
                       compiler="qiskit",
                       compiler_settings=compiler_settings,
                       provider_name="ionq",)
    circuits.append((i,qc))
    print("---------------vqe with qubit", i,"---------------")
print("=================================================================================================")
print(circuits)

---------------vqe with qubit 3 ---------------
---------------vqe with qubit 4 ---------------
[(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x72c393df4800>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x72c421c51c70>)]


In [2]:
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

simulator = AerSimulator()

jobs = []
for qubit_number, circuit_object in circuits:
    # Run noisy simulation (counts)
    job = simulator.run(circuit_object, shots=1000)
    results = job.result()
    counts = results.get_counts()

    # Get ideal distribution from statevector
    qc_nom = circuit_object.remove_final_measurements(inplace=False)
    state = Statevector.from_instruction(qc_nom)
    ideal_probs = state.probabilities_dict()

    jobs.append((qubit_number, counts, ideal_probs))

print("Collected results:")
for res in jobs:
    print(res)


Collected results:
(3, {'110': 1, '011': 18, '100': 6, '101': 29, '010': 7, '001': 939}, {'000': 1.646305228181368e-11, '001': 0.9354135590938883, '010': 0.009983617727545434, '011': 0.014098485600205369, '100': 0.00410394440945187, '101': 0.03599113874463008, '110': 0.0004092532498016922, '111': 1.158013847925622e-09})
(4, {'1001': 1000}, {'0000': 1.1939127850311922e-08, '0001': 4.757422743792308e-10, '0010': 1.6505000563282705e-08, '0011': 3.877382423926261e-09, '0100': 3.2619881386380706e-16, '0101': 7.385748123608895e-17, '0110': 1.32366079729259e-16, '0111': 5.1290945453411586e-17, '1000': 5.143090961236338e-09, '1001': 0.9999998997629683, '1010': 5.413734704979609e-08, '1011': 2.708666539585789e-10, '1100': 2.4547065430724234e-09, '1101': 2.752051277847019e-09, '1110': 2.5684129745521114e-09, '1111': 1.1330228023536813e-10})


In [3]:
# Hellinger fidelity function
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
       and expected distribution q (probabilities)."""
    p_sum = sum(p.values())
    p_norm = {k: v / p_sum for k, v in p.items()}

    q_sum = sum(q.values())
    q_norm = {k: v / q_sum for k, v in q.items()}

    overlap = 0.0
    for k in set(p_norm) | set(q_norm):
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))
    return overlap**2

In [4]:
for qubit_number, counts, ideal_probs in jobs:
    fidelity = hellinger_fidelity_with_expected(counts, ideal_probs)
    print(f"Hellinger Fidelity (VQE, {qubit_number} qubits): {fidelity:.4f}")

Hellinger Fidelity (VQE, 3 qubits): 0.9988
Hellinger Fidelity (VQE, 4 qubits): 1.0000


In [5]:
for qubit_number, counts, ideal_probs in jobs:
    fidelity = hellinger_fidelity_with_expected(counts, ideal_probs)
    print(f"Hellinger Fidelity (VQE, {qubit_number} qubits): {fidelity:.4f}")

Hellinger Fidelity (VQE, 3 qubits): 0.9988
Hellinger Fidelity (VQE, 4 qubits): 1.0000
